<a href="https://colab.research.google.com/github/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/03-comp-analise-semantica-cmmsemantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Semântica

A Análise Semântica consiste em uma "segunda passada" do compilador no código, pois o código fonte é novamente analisado desde o início na **AST**, porém agora para realizar a anotação de atributos, inferência de tipos e verificações se as construções sintáticas fazem algum sentido para a linguagem.

## Introdução

Como entrada para a _Análise Semântica_ temos a Árvore Sintática Abstrata (AST) produzida pela fase anterior, a _Análise Sintática_ com a utilização da lista de _tokens_ identificados pela _Análises Léxica_.

Nesta terceira parte do desenvolvimento do projeto do Compilador, a **AST** deve ser percorrida para a realização da análise sensível ao contexto e a geração ou finalização da _Tabela de Símbolos_. 

O principal motivo da **Análise Semântica** é verificar se existem erros de contexto para a linguagem **C-** apresentada na primeira e segunda parte deste tutorial. Um conjunto de Regras Semânticas devem ser verificadas.

## Preparação do Ambiente



In [ ]:
!pip install ply
!pip install anytree
!pip install graphviz
!pip install llvmlite
!jupyter nbextension install https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip  --user
!jupyter nbextension enable highlighter/highlighter

Downloading: https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip -> /tmp/tmpys_gg4/highlighter.zip
Extracting: /tmp/tmpys_gg4/highlighter.zip -> /root/.local/share/jupyter/nbextensions
Enabling notebook extension highlighter/highlighter...
      - Validating: OK


## Copiando Módulos do Projeto

Aqui iremos copiar o projeto `rea-comp04-compiladores` do repositório do [`github`](https://github.com/rogerioag/rea-comp04-compiladores):

```bash
git clone https://github.com/rogerioag/rea-comp04-compiladores.git
```
Iremos copiar os módulos e os arquivos de testes para o diretório de conteúdo do _notebook_, o __content__.


In [ ]:
! rm -rf rea-comp04-compiladores
! git clone https://github.com/rogerioag/rea-comp04-compiladores.git
! cp -R rea-comp04-compiladores/cmmcompiler/* .
! cp -R rea-comp04-compiladores/cmmcompiler/tests/* .


Cloning into 'rea-comp04-compiladores'...
remote: Enumerating objects: 470, done.
remote: Counting objects: 100% (470/470), done.
remote: Compressing objects: 100% (370/370), done.
remote: Total 470 (delta 264), reused 234 (delta 89), pack-reused 0
Receiving objects: 100% (470/470), 2.87 MiB | 2.55 MiB/s, done.
Resolving deltas: 100% (264/264), done.


## Leitura Recomendada

1. __Capítulo 6:__ _Análise Semântica_

    LOUDEN, Kenneth C. Compiladores: princípios e práticas. São Paulo, SP: Thomson, c2004. xiv, 569 p. ISBN 8522104220.

3. __Capítulo 5:__ _Tradução Dirigida pela Sintaxe_

    AHO, Alfred V. et al. Compiladores: princípios, técnicas e ferramentas. 2. ed. São Paulo, SP: Pearson Addison-Wesley, 2008. x, 634 p. ISBN 9788588639249.


## Regras Semânticas

As **Regras Semânticas** que precisam ser verificadas para a identicação de erros semânticos ou de contexto são:

### **Funções e Procedimentos.** 

Cada um dos _identificadores de função_, nome e quantidade de parâmetros formais deve ser verificado, os parâmetros formais devem estar vinculados a identificadores de variáveis locais.
 
1. **Função Principal.** Todo programa escrito em **C-** deve ter uma *função principal* declarada, com a identificação `main`. Verificar a existência de uma função principal que inicializa a execução do código. Caso contrário, deve apresentar a mensagem:

  **Erro:** *Função principal não declarada.*

2. A função principal normalmente é do tipo `int`, mas também é aceito o retorno do tipo `void`, assim é esperado que seu retorno seja um valor inteiro ou vazio, do contrário a mensagem deve ser emitida:
	
	**Erro:** Função principal deveria retornar valor inteiro ou vazio.

3. A quantidade de parâmetros reais de uma chamada de função/procedimento `func` deve ser igual a quantidade de parâmetros formais da sua definição. Caso contrário, gerar a mensagem:
	
  **Erro:** Chamada à função 'func' com número de parâmetros menor que o declarado.

4. Uma função deve retornar um valor de tipo compatível com o tipo de retorno declarado. Se a função `main` que é declarada com retorno `int`, não apresenta um `return(0)`, a mensagem deve ser gerada:

  **Erro:** Função 'main' deveria retornar inteiro, mas retorna vazio.

5. Funções precisam ser declaradas antes de serem chamadas/ativadas. Caso contrário a mensagem de erro deve ser emitida: 
   
  **Erro:** Chamada a função 'func' que não foi declarada.

6. Uma função qualquer não pode fazer uma chamada à função `main`. Devemos verificar ser existem alguma chamada para a função `main` partindo de qualquer outra função do programa.
   
  **Erro:** Chamada para a função principal não permitida.

7. Uma função pode ser declarada e não utilizada. Se isto acontecer uma aviso deverá ser emitido:
	
  **Aviso:** Função 'func' declarada, mas não utilizada.

8. Se a função `main` fizer uma chamada para ela mesmo, a mensagem de aviso deve ser emitida:
	
  **Aviso:** Chamada recursiva para 'main'.

### Variáveis

Os __identificadores de variáveis locais e globais__: nome, tipo e escopo devem ser aramazenados na Tabela de Símbolos.
   
9. Variáveis devem ser **declaradas**, **inicializadas** antes de serem **utilizadas** (leitura).
   Lembrando que uma variável pode ser declarada:
    a. no escopo do procedimento (como expressão ou como parâmetro formal);
    b. no escopo global

    *Warnings*/Avisos deverão ser mostrados quando uma variável for declarada mas nunca utilizada. 
  
10. Se uma variável 'a' for apenas declarada e não for inicializada (escrita) ou não for utilizada (não lida), o analisador deve gerar a mensagem:
	
    **Aviso:** Variável 'a' declarada e não utilizada.
	
11. Se houver a tentativa de leitura ou escrita de qualquer variável não declarada a seguinte mensagem:
	
    **Erro:** Variável 'a' não declarada.
	
12. Se houver a tentativa de leitura de uma variável 'a' declarada, mas não inicializada:
	
    **Aviso:** Variável 'a' declarada e não inicializada.
	
13. *Warnings* deverão ser mostrados quando uma variável for declarada mais de uma vez. Se uma variável 'a' for declarada duas vezes no mesmo escopo, o aviso deve ser emitido:
	
    **Aviso:** Variável 'a' já declarada anteriormente

### **Atribuição.**

Na atribuição devem ser verificados se os tipos são compatíveis. Por exemplo, uma variável `a` recebe uma expressão `b + c`. Os tipos declarados para `a` e o tipo resultado da inferência do tipo da expressão `b + c` deverão ser compatíveis. Se `b` for `int` e `c` for `int`, o tipo resultante da expressão será também `int`.

14. Se assumirmos, por exemplo, que `b` é do tipo `int` e `c` do tipo `float`, o resultado pode ser `float` (se assumirmos isso para nossa linguagem). O que faria a atribuição `a := b + c` apresentar tipos diferentes e a mensagem deve ser apresentada:
	
    **Aviso:** Atribuição de tipos distintos: 'a' int e 'expressão' float

  O mesmo pode acontecer com a atribuição de um retorno de uma função, se os tipos forem incompatíveis o usuário deve ser avisado:
	
    **Aviso:** Atribuição de tipos distintos 'a' float e 'func' retorna int

15. **Coerções implícitas.** *Warnings* deverão ser mostrados quando ocorrer uma coerção implícita de tipos (int<->float). Atribuição de variáveis ou resultados de expressões de tipos distintos devem gerar a mensagem:
	
    **Aviso:** Coerção implícita do valor de 'x'.
    **Aviso:** Coerção implícita do valor retornado por 'func'.

### **Arranjos.**

Na linguagem `C-` é possível declarar arranjos, pela sintaxe da linguagem o índice de um arranjo é inteiro e isso deve ser verificado. Na tabela de símbolos devemos armazenar se uma variável declarada tem um tipo, se é uma variável escalar ou um vetor ou uma matriz. Podemos armazenar um campo 'dimensões', que '0': escalar, '1': arranjo unidimensional (vetor) e '2': arranjo bidimensional (matriz) e assim por diante.

16. Encontrado a referência a um arranjo, seu o índice, seja um número, variável ou expressão deve ser um `inteiro`. Do contrário, a mensagem deve ser gerada: 
	
    __Erro:__ Índice de array 'X' não inteiro.

17. Se o acesso ao elemento do arranjo estiver fora de sua definição, por exemplo um vetor `A` é declarado como tendo `10` elementos (0 a 9) e há um acesso ao `A[10]`, a mensagem de erro deve ser apresentada: 
	
    **Erro:** índice de array 'A' fora do intervalo (out of range)

18. E outros situações descritas nos arquivos de testes.



## Tabela de Símbolos

Para a realização da análise dos erros apontados, a construção da **Tabela de Símbolos** deve ser realizada adequadamente. Uma Tabela de Símbolos pode começar a ser construída desde as análises anteriores. Por exemplo, na **Análise Léxica**, onde temos os `TOKENS` e os lexemas que casaram com a expressão regular que gerou cada um desses _tokens_. Temos também o número da `linha` e da `coluna` que o lexema foi encontrado no código fonte de entrada. 

Desta forma, as entradas da Tabela de Símbolos podem ir sendo preenchidas com essas informações iniciais <TOKEN, LEXEMA, LINHA, COLUNA>.

Na **Análise Sintática** quando o reconhecimento entrar na função que trata a regra, por exemplo, na regra de _declaração de variáveis_, todas as variáveis irão assumir o tipo declarado para cada uma delas.

``int a;``
``int b;
``int c[10]``

Que entraria na regra:

``var-declaration ::= type-specifier ID ; | type-specifier ID [ NUM ] ; ``

A declaração indica que as variáveis escalares `a` e `b` e o arranjo unidimensional `c` terão por declaração o `tipo` que no exemplo é `int`.

## Exemplo

```.c
int a;
int b[10];
int c[3][5];

b = 10

int func(int x, int y){
  int res;
  res = x + y;
  return(res);
}

int main(){
  a = input();
  b = input();

  b[0] = a;
  b[1] = b;
  c[0][1] = func(a,b);

  output(c[3]);
  return(0);
}
  
```


| Token  | Lexema  | Tipo  |	dim  | tam_dim1 | tam_dim2  | escopo  | init  | linha  |  coluna  |
| :----: | :-----: | :----: | :-----| :--- | :------ | :---- | :----- | :------- | ------|
|ID|"a"|int|0|1|0|global|N|1|5|
|ID|"b"|int|1|10|0|global|N|2|5|
|ID|"c"|int|2|3|5|global|N|3|5|




## Árvore Sintática Abstrata

Após a análise semântica e geração de dados referentes a esta passagem, é esperado como saída uma árvore sintática abstrata anotada (ASTO).

Tomemos como exemplo uma atribuição ``a := b + c``, de acordo com a sintaxe da linguagem **TPP** temos a subárvore representada na _Figura 3.1_.

<img src="https://raw.githubusercontent.com/rogerioag/rea-comp04-compiladores/main/jupyter-notebooks/figuras/atribuicao.png" alt="Atribuição expandida" width="55%"/>

__Figura 3.1:__ Atribuição expandida

Após a verificação das regras semânticas, pode-se fazer uma poda dos nós internos da árvore para facilitar a geração de código. A _Figura 3.2_ apresenta a Árvore Sintática Abstrata simplificada.

<img src="https://raw.githubusercontent.com/rogerioag/rea-comp04-compiladores/main/jupyter-notebooks/figuras/atribuicao-asa.png" alt="Atribuição depois da poda" width="25%"/>

__Figura 3.2:__ Atribuição depois da poda

A simplificação da árvore eliminando os nós intermediários facilitará a geração de código. O Código que deve ser gerado para uma ``atribuicao`` é apresentado no Código:

```llvm
  %2 = load i32, i32* @b, align 4
  %3 = load i32, i32* @c, align 4
  %4 = add nsw i32 %2, %3
  store i32 %4, i32* @a, align 4
```



## Testes

Alguns casos de testes estão disponíveis no conjunto de testes no projeto [`rea-comp04-compiladores`](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/tests).



## Referências

[1] LOUDEN, Kenneth C. **Compiladores:** princípios e práticas. São Paulo, SP: Thomson, c2004. xiv, 569 p. ISBN 8522104220.

[2] AHO, Alfred V. **Compiladores:** princípios, técnicas e ferramentas. 2. ed. São Paulo: Pearson Addison-Wesley, 2008. x, 634 p. ISBN 9788588639249.